## ЗАДАНИЕ 12.2. ПРАКТИКА 2

*выполнено Натальей Лисицкой*

**Задание. Добавьте к атрибутам количество комментариев по новости.**

In [1]:
import datetime
from time import sleep, time

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import csv
from pathlib import Path

In [9]:
filename = "articles_info.csv" # имя файла, в который будем сохранять результат
driver_path = r'C:\Program Files (x86)\Google\Chrome\chromedriver.exe' # укажите ваш путь к chromedriver, который вы загрузили ранее
base_dir= r"F:\SkillFactory\Y2_U6" # укажите директорию, в которую будем сохранять файл
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36 OPR/83.0.4254.27" #Ваш user-agent. Узнать можно тут https://юзерагент.рф, смотреть с браузера Chrome
start_time = time() # время начала выполнения программы
#url = 'https://news.ycombinator.com'

In [10]:
def get_load_time(article_url, user_agent):
    #будем ждать 3 секунды, иначе выводить exception и присваивать константное значение
    try:
        # меняем значение заголовка. По умолчанию указано, что это python-код
        headers = {
            "User-Agent": user_agent
        }
        # делаем запрос по url статьи article_url
        response = requests.get(
            article_url, headers=headers, stream=True, timeout=3.000
        )
        # получаем время загрузки страницы
        load_time = response.elapsed.total_seconds()
    except Exception as e:
        print(e)
        load_time = ">3"
    return load_time

In [11]:
def write_to_file(output_list, filename, base_dir):
    for row in output_list:
        with open(Path(base_dir).joinpath(filename), "a") as csvfile:
            fieldnames = ["id", "load_time", "rank", "points", "title", "url", "comments"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow(row)

In [12]:
def connect_to_base(browser, page_number):
    base_url = "https://news.ycombinator.com/news?p={}".format(page_number)
    for connection_attempts in range(1,4): # совершаем 3 попытки подключения
        try:
            browser.get(base_url)
            # ожидаем пока элемент table с id = 'hnmain' будет загружен на страницу
            # затем функция вернет True иначе False 
            WebDriverWait(browser, 5).until(
                EC.presence_of_element_located((By.ID, "hnmain"))
            )
            return True
        except Exception as e:
            print(e)
            print("Error connecting to {}.".format(base_url))
            print("Attempt #{}.".format(connection_attempts))
    return False

In [13]:
def parse_html(html, user_agent):
    soup = BeautifulSoup(html, "html.parser")
    output_list = []
    
    # ищем в объекте soup object id, rank, score и title статьи
    tr_blocks = soup.find_all("tr", class_="athing")
    article = 0
    for tr in tr_blocks:
        article_id = tr.get("id") # id
        article_url = tr.find_all("a")[1]["href"]
        
        '''
        Поиск количества комментариев по нововсти:
        - переход к следующей стоке, в ней будем искать количество комментариев
        - в этой строке ищем все тэги со ссылками (нам нужен последний)
        - берем текст от последнего тэга
        '''
        
        tr_next = tr.next_sibling 
        #print(tr_next)
        try:
            #comment_link = tr_next.find_element_by_partial_link_text('comment')
            comment_link = tr_next.find_all('a')
            comments = comment_link[-1].text
        except Exception as e:
            print('comments', e)
            comments = "0 comments"

        # иногда статья располагается не на внешнем сайте,
        #а на ycombinator, тогда article_url у нее не полный, а добавочный, 
        #с параметрами. Например item?id=200933. Для этих случаев будем добавлять урл до полного
        
        if "item?id=" in article_url or "from?site=" in article_url:
            article_url = f"https://news.ycombinator.com/{article_url}"
        load_time = get_load_time(article_url, user_agent)
        # иногда рейтинга может не быть, поэтому воспользуемся try

        try:
            score = soup.find(id=f"score_{article_id}").string
        except Exception as e:
            print('score', e)
            score = "0 points"
        '''
        Для rank, title также сделаем обработку ошибок
        '''
        try:
            rank = tr.span.string
        except Exception as e:
            print('rank', e)
            rank = ""
        
        try:
            title = tr.find(class_="titlelink").string
        except Exception as e:
            print('title', e)
            title = ""
            
        article_info = {
            "id": article_id,
            "load_time": load_time,
            "rank": rank,
            "points": score,
            "title": title,
            "url": article_url,
            "comments": comments,
        }

        # добавляем информацию о статье в список
        output_list.append(article_info)
        article += 1
    return output_list

### ОДНОПОТОЧНОСТЬ

In [15]:
# инициализируем веб драйвер
browser = webdriver.Chrome(executable_path=driver_path)

# перебираем страницы и собираем нужную информацию
for page_number in range(10):
    print("getting page " + str(page_number) + "...")
    if connect_to_base(browser, page_number):
        sleep(5)
        output_list = parse_html(browser.page_source, user_agent)
        write_to_file(output_list, filename, base_dir)

    else:
        print("Error connecting to hacker news")
    
# завершаем работу драйвера
browser.close()
sleep(1)
browser.quit()
end_time = time()
elapsed_time = end_time - start_time
print("run time: {} seconds".format(elapsed_time))

getting page 0...
('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None))
score 'NoneType' object has no attribute 'string'
getting page 1...
('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None))
score 'NoneType' object has no attribute 'string'
getting page 2...
getting page 3...
HTTPSConnectionPool(host='www.quantamagazine.org', port=443): Read timed out. (read timeout=3.0)
getting page 4...
getting page 5...
('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None))
getting page 6...
getting page 7...
getting page 8...
HTTPSConnectionPool(host='en.wikipedia.org', port=443): Read timed out. (read timeout=3.0)
getting page 9...
HTTPConnectionPool(host='www.msen.com', port=80): Max retries exceeded with url: /~clif/TclTutor.html (Caused by Conne

### МНОГОПОТОЧНОСТЬ

In [16]:
from concurrent.futures import ThreadPoolExecutor, wait

def run_process(page_number, filename):
    browser = webdriver.Chrome(executable_path=driver_path)
    if connect_to_base(browser, page_number):
        sleep(5)
        output_list = parse_html(browser.page_source, user_agent)
        write_to_file(output_list, filename, base_dir)
        
        browser.quit()
    else:
        print("Error connecting to hacker news")
        browser.quit()

In [17]:
start_time = time()

futures = []

with ThreadPoolExecutor() as executor:
    for number in range(10):
        futures.append(
            executor.submit(run_process, number, filename)
        )
wait(futures)
end_time = time()
elapsed_time = end_time - start_time
print("Elapsed run time: {} seconds".format(elapsed_time))

Message: 

Error connecting to https://news.ycombinator.com/news?p=6.
Attempt #1.
Message: 

Error connecting to https://news.ycombinator.com/news?p=7.
Attempt #1.
Message: 

Error connecting to https://news.ycombinator.com/news?p=9.
Attempt #1.
HTTPSConnectionPool(host='m.rbi.org.in', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='m.rbi.org.in', port=443): Read timed out. (read timeout=3.0)
score 'NoneType' object has no attribute 'string'
score 'NoneType' object has no attribute 'string'
('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None))
Elapsed run time: 54.77807426452637 seconds
